In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sqlalchemy import create_engine

df = pd.read_csv('movies_titles.csv')
df.head()

,show_id,type,title,director,cast,country,release_year,rating,duration,description,...,Language TV Shows,Musicals,Nature TV,Reality TV,Spirituality,TV Action,TV Comedies,TV Dramas,Talk Shows TV Comedies,Thrillers
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2020,PG-13,90 min,As her father nears the end of his life filmma...,...,0,0,0,0,0,0,0,0,0,0
1,s2,TV Show,Blood & Water,NaN,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,...,0,0,0,0,0,0,0,1,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha ...,NaN,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,...,0,0,0,0,0,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021,TV-MA,1 Season,Feuds flirtations and toilet talk go down amon...,...,0,0,0,1,0,0,0,0,0,0
4,s5,TV Show,Kota Factory,NaN,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,...,0,0,0,0,0,0,1,0,0,0


In [2]:
df.isna().sum()
df.shape

(8508, 43)

### Create the TFID vecoritzer and take out the stopwords

In [3]:
# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df['description'])

# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)

# Preview the matrix by placing it into a DataFrame (which we won't need later)
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df['description'])
df_tfidf

(8508, 18568)


description,As her father nears the end of his life filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.,After crossing paths at a party a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.,To protect his family from a powerful drug lord skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.,Feuds flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.,In a city of coaching centers known to train India’s finest collegiate minds an earnest but unexceptional student and his friends navigate campus life.,The arrival of a charismatic young priest brings glorious miracles ominous mysteries and renewed religious fervor to a dying town desperate to believe.,Equestria's divided. But a bright-eyed hero believes Earth Ponies Pegasi and Unicorns should be pals — and hoof to heart she’s determined to prove it.,On a photo shoot in Ghana an American model slips back in time becomes enslaved on a plantation and bears witness to the agony of her ancestral past.,A talented batch of amateur bakers face off in a 10-week competition whipping up their best dishes in the hopes of being named the U.K.'s best.,A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.,...,Teen surfer Zak Storm is mysteriously transported to the Bermuda Triangle where he becomes the captain of a magical ship full of misfits.,A philandering small-town mechanic's political ambitions are sparked when the visiting prime minister mistakenly grants him special security clearance.,A change in the leadership of a political party sparks bitter conflict and the party's division into two rival factions.,Strong-willed middle-class Kashaf and carefree wealthy Zaroon meet in college but before love can take root they each have some growing up to do.,Recovering alcoholic Talal wakes up inside a small-town police station cell where he's subject to the mind games of a psychotic sadist.,A political cartoonist a crime reporter and a pair of cops investigate San Francisco's infamous Zodiac Killer in this thriller based on a true story.,While living alone in a spooky town a young girl befriends a motley crew of zombie children with diverse personalities.,Looking to survive in a world taken over by zombies a dorky college student teams with an urban roughneck and a pair of grifter sisters.,Dragged from civilian life a former superhero must train a new crop of youthful saviors when the military preps for an attack by a familiar villain.,A scrappy but poor boy worms his way into a tycoon's dysfunctional family while facing his fear of music and the truth about his past.
007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239072,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ömer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
über,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
łukasz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ōarai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the cosine similarity scores and put them in a df

In [4]:
# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# For easier viewing, put it in a dataframe
df_results = pd.DataFrame(cosine_sim, columns=df['show_id'], index = df['show_id'])
# df_results.to_csv("content_filtering_recommendations.csv")
df_results.sort_values(by=['s8'], ascending=False)
# df_results

show_id,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s8798,s8799,s8800,s8801,s8802,s8803,s8804,s8805,s8806,s8807
show_id,,,,,,,,,,,,,,,,,,,,,
s8,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.035930
s8354,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.148420,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
s5581,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.124839,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
s5654,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.119943,0.000000,0.035672,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.037600
s3620,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.114454,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.032855,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s3014,0.000000,0.0,0.053262,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
s3013,0.095163,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.045265,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
s3012,0.000000,0.0,0.000000,0.022627,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000


### Build the csv file that holds all the recommendations

In [5]:
# Create a DataFrame to hold the top 20 recommendations for each show
top_n = 20
top_recommendations = {}

for show_id in df['show_id']:
    similar_scores = df_results[show_id].drop(show_id)  # Drop self-match
    top_similar = similar_scores.sort_values(ascending=False).head(top_n).index.tolist()
    top_recommendations[show_id] = top_similar

# Convert to a DataFrame
recommendations_df = pd.DataFrame.from_dict(top_recommendations, orient='index')
recommendations_df.index.name = 'show_id'
recommendations_df = recommendations_df.reset_index()

In [7]:
recommendations_df

,show_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,s1,s4878,s1067,s7507,s5048,s5234,s5495,s2675,s4242,s2761,...,s1732,s2430,s2381,s7366,s6387,s1947,s5798,s4156,s4087,s8701
1,s2,s5345,s1885,s4286,s4272,s4210,s614,s109,s6290,s1906,...,s6714,s6258,s4488,s6830,s7639,s6053,s4384,s2002,s8800,s8783
2,s3,s425,s5114,s6593,s5306,s3452,s712,s1906,s4032,s3977,...,s8570,s3298,s2088,s6210,s2550,s1350,s3553,s8018,s3843,s3024
3,s4,s2133,s8465,s4710,s8467,s1392,s1466,s7475,s2346,s3838,...,s7868,s6361,s8756,s8250,s3238,s1292,s2395,s5306,s8013,s3364
4,s5,s2273,s267,s2092,s4077,s806,s2691,s8233,s8613,s2363,...,s88,s3462,s8493,s1445,s2083,s3131,s5832,s4921,s6231,s3490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8503,s8803,s5953,s201,s4454,s6380,s3091,s6213,s3066,s2221,s7342,...,s6311,s3616,s5379,s3711,s3967,s2139,s5958,s3771,s3788,s3580
8504,s8804,s2972,s7287,s2882,s6250,s7461,s4810,s7840,s8484,s1718,...,s6263,s1893,s8396,s5442,s8090,s1880,s1949,s5004,s794,s518
8505,s8805,s8637,s1842,s1632,s8112,s2092,s3912,s1265,s3651,s2964,...,s2809,s1409,s1460,s5154,s770,s1299,s4504,s5941,s2473,s2834
8506,s8806,s8068,s2404,s6294,s8584,s6327,s8613,s2857,s88,s3293,...,s5267,s54,s7089,s4319,s1351,s562,s2738,s103,s1636,s1405


### put it in the Azure database

In [6]:
# Define your Azure SQL connection details
server = '1-10intex.database.windows.net'
database = 'Movies'
username = 'pigadmin'
password = 'Superduperstrongpassword!'
driver = 'ODBC Driver 18 for SQL Server'  # Or another valid installed driver

# Encode the driver string for URL
driver_encoded = driver.replace(' ', '+')

# Create the connection URL
connection_string = f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={driver_encoded}&Encrypt=yes&TrustServerCertificate=no&Connection Timeout=30'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Upload DataFrame to Azure
recommendations_df.to_sql(
    'content_filtering_recommendations',
    con=engine,
    if_exists='replace',  # or 'append' if you want to add new data without replacing
    index=False,
    chunksize=1000
)

93